### Gates

A Gate is an effect that can be applied directly to the Qubits, A Gate represents physical process that can be applied to the Qubits, alternately by calling the gates on Qubits. The object created by calling the gates on Qubits is called an Operation. A gate can be thought of an input Qubit and generates an associated GateOperation object.

Gates can generally be applied to any type of qubit (NamedQubit, LineQubit, GridQubit, etc) to create an Operation. However, depending on the application, you may prefer a specific type of qubit. For instance, Google devices generally use GridQubits. Other devices may have connectivity constraints that further restrict the set of qubits that can be used, especially in multi- qubit operations.

In [1]:
## This section covers the three qubit quantum teleportation gate

import cirq

# This example uses named Qubits to remain abstract
# We can also use line qubits or grid qubits to specify a geometry
a = cirq.NamedQubit('a')
b = cirq.NamedQubit('b')
c = cirq.NamedQubit('c')

# Example Operations, that correspond to the moments above
print(cirq.H(b))
print(cirq.CNOT(b, c))
print(cirq.CNOT(a, b))
print(cirq.H(a))
print(cirq.measure(a,b))


# prints
# H(b)
# CNOT(b, c)
# CNOT(a, b)
# H(a)
# cirq.MeasurementGate(2, 'a,b', ())(a, b)

H(b)
CNOT(b, c)
CNOT(a, b)
H(a)
cirq.MeasurementGate(2, 'a,b', ())(a, b)


In [2]:
## At the base of this construction is the notion of a qubit. In Cirq, qubits and other quantum objects are identified by instances of subclasses of the Qid base class. Different subclasses of Qid can be used for different purposes. For example, the qubits that Google’s Xmon devices use are often arranged on the vertices of a square grid. For this, the class GridQubit subclasses Qid. For example, we can create a 3 by 3 grid of qubits using
qubits = [cirq.GridQubit(x, y) for x in range(3) for y in range(3)]

print(qubits[0])

(0, 0)


In [3]:
## Simple Gate
x_gate = cirq.X
# Applying it to the qubit at location (0,0) in the qubits in the above cell
# Turns it into an operation
x_op = x_gate(qubits[0])

print(x_op)

X((0, 0))


In [5]:
cz = cirq.CZ(qubits[0], qubits[1])
x =  cirq.X(qubits[2])
moment = cirq.Moment([x, cz])

print(moment)

## A moment is a collection of operation on a disjointed set of Qubits

X((0, 2)) and CZ((0, 0), (0, 1))


In [ ]:
### This program illustrates creation of Two moments on CZ Gates on three different qubits
cz01 = cirq.CZ(qubits[0], qubits[1])
cz12 = cirq.CZ(qubits[1], qubits[2])
x2 = cirq.X(qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0, moment1))

print(circuit)

### Constructing Circuits

Constructing Circuits as a series of Moment objects, with each Moment being hand-crafted, is tedious. Instead, we provide a variety of different ways to create a Circuit.

One of the most useful ways to construct a Circuit is by appending onto the Circuit with the Circuit.append method.

In [5]:
from cirq.ops import CZ, H
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]
cirquit = cirq.Circuit()
cirquit.append([CZ(q0, q1), H(q2)])

print(cirquit)

(0, 0): ───@───
           │
(1, 0): ───@───

(2, 0): ───H───


In [8]:
cirquit.append([H(q0), CZ(q1, q2)])

print(cirquit)

(0, 0): ───@───H───H───
           │
(1, 0): ───@───@───@───
               │   │
(2, 0): ───H───@───@───


### Insert Strategy
InsertStrategy defines how Operations are placed in a Circuit when requested to be inserted at a given location. Here, a location is identified by the index of the Moment (in the Circuit) where the insertion is requested to be placed at (in the case of Circuit.append, this means inserting at the Moment, at an index one greater than the maximum moment index in the Circuit).

There are four such strategies: InsertStrategy.EARLIEST, InsertStrategy.NEW, InsertStrategy.INLINE and InsertStrategy.NEW_THEN_INLINE, InsertStrategy.EARLIEST is defined as:

Scans backward from the insert location until a moment with operations touching qubits affected by the operation to insert is found. The operation is added to the moment just after that location.

For example, if we first create an Operation in a single moment, and then use InsertStrategy.EARLIEST, Operation can slide back to this first Moment if there is space:

In [9]:
from cirq.circuits import InsertStrategy
circuit = cirq.Circuit()
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)

print(circuit)

(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


In [10]:
###Attempts to add the operation to insert into the moment just before the desired insert location. But, if there’s already an existing operation affecting any of the qubits touched by the operation to insert, a new moment is created instead.
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.INLINE)

print(circuit)

(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


In [11]:
##Creates a new moment at the desired insert location for the first operation, but then switches to inserting operations 
##according to ``InsertStrategy.INLINE``.
circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1,q2), H(q0)], strategy=InsertStrategy.NEW_THEN_INLINE)

print(circuit)

(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


## Patterns for Arguments to Append and Insert

In the above examples, we used a series of Circuit.appendcalls with a list of different Operations added to the circuit. However, the argument where we have supplied a list can also take more than just list values. For instance:


In [13]:

def my_layer():
    yield CZ(q0, q1)
    yield [H(q) for q in (q0, q1, q2)]
    yield [CZ(q1, q2)]
    yield [H(q0), [CZ(q1, q2)]]

circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)
    
print(circuit)

CZ((0, 0), (1, 0))
[cirq.H(cirq.GridQubit(0, 0)), cirq.H(cirq.GridQubit(1, 0)), cirq.H(cirq.GridQubit(2, 0))]
[cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H(cirq.GridQubit(0, 0)), [cirq.CZ(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]
(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───
